In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Res34SimSiam
from dataset import Dataset_ori
from torch.utils.data import DataLoader
import numpy as np
from collections import OrderedDict

In [22]:
def test_epoch(MODEL_PATH, test_loader):
    
    state_dict = torch.load(MODEL_PATH) 
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] #remove 'module'
        new_state_dict[name] = v

    state_dict = new_state_dict
    model = Res34SimSiam(512, 128).cuda()
    model.load_state_dict(state_dict)

    with torch.no_grad():

        signal_preds = []
        signal_pred_probs = None
        all_targets = None
        
        model.eval()

        for batch_idx, (signal, _) in enumerate(test_loader):

            signal = signal.cuda().float()
            
            _, _, _, _, _, signal_out = model(signal, signal)
            
            signal_predicted = signal_out.argmax(1)
            signal_preds.append(signal_predicted.detach().cpu().numpy())
            
    return np.concatenate(signal_preds)

In [23]:
x_path = '/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy'
y_path = '/labs/hulab/Robust_learning_TESTDATA/label_simband.npy'
data_x = np.load(x_path)
data_y = np.load(y_path)
print(data_x.shape, data_y.shape)

(857, 2400) (857,)


In [25]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt'
test_dataset = Dataset_ori(x_path, y_path)
testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
predictions = test_epoch(MODEL_PATH, testloader)
print(predictions.shape)
print(predictions)

(857,)
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1